# WISE2MBH Tutorial: WXSC

This sample contain WISE data from the WXSC catalog. Distances, morphologies and photometric/physical quantities are provided. At the moment of calculating W1 absolute magnitude, the distance modulus is calculated from the provided distance.

In [1]:
import wise2mbh as wm
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import random
no_data = 9876543

Could not import regions, which is required for some of the functionalities of this module.


In [18]:
wxsc = Table.read('../samples/WXSC_sample.fit')
wxsc

Name,RAJ2000,DEJ2000,Morph,b_a,PA,R1,W1mag,R2,W2mag,R3,W3mag,R4,W4mag,Dist,Beam,W1-W2,W2-W3,LogLw1,LogMs,LognuL3,LognuL4,SFR12,SFR23,sSFR12,RW1,AW1mag,e_AW1mag,RW2,AW2mag,e_AW2mag,RW3,AW3mag,e_AW3mag,RW4,AW4mag,e_AW4mag,TW1mag,e_TW1mag,TW2mag,e_TW2mag,TW3mag,e_TW3mag,TW4mag,e_TW4mag,SimbadName,NED,recno
,deg,deg,,,deg,arcmin,mag,arcmin,mag,arcmin,mag,arcmin,mag,Mpc,pc,mag,mag,log(solLum),log(solMass),log(solLum),log(solLum),solMass / yr,solMass / yr,1 / yr,arcmin,mag,mag,arcmin,mag,mag,arcmin,mag,mag,arcmin,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,
bytes11,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes20,bytes3,int32
LMC,79.64101,-68.81836,SBm,1.0,0.0,300.0,-1.93,300.0,-1.99,300.0,-4.78,300.0,-7.15,0.05,1,0.06,2.79,9.47,9.15,7.62,7.76,0.12,0.03,-10.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LMC,NED,1
SMC,13.1866,-72.8286,SBm-pec,1.0,0.0,166.67,0.56,166.67,0.62,166.67,-0.93,166.67,-3.48,0.06,2,-0.05,1.55,8.63,8.59,6.06,6.43,0.01,0.01,-10.59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SMC,NED,2
MESSIER31,10.68479,41.26907,Sb,0.35,37.3,111.44,0.08,111.44,0.13,111.44,-1.95,111.44,-3.09,0.77,22,-0.03,2.09,11.04,10.95,8.82,8.48,1.2,0.58,-10.88,149.39,0.08,0.01,132.78,0.13,0.01,148.8,-1.95,0.01,99.38,-3.11,0.01,0.06,0.01,0.1,0.01,-2.0,0.01,-3.13,0.01,MESSIER31,NED,3
MESSIER33,23.46204,30.66022,Sc,0.59,18.9,31.64,3.09,31.64,3.02,31.64,-0.17,31.64,-1.94,0.86,25,0.07,3.18,9.97,9.61,8.33,8.22,0.45,0.33,-9.96,65.32,3.03,0.01,38.83,3.0,0.01,37.28,-0.17,0.01,30.55,-1.94,0.01,3.08,0.01,3.02,0.01,-0.22,0.01,-2.02,0.01,MESSIER33,NED,4
NGC0253,11.88844,-25.28827,SABc,0.28,53.0,21.13,3.46,21.13,3.27,21.13,-0.54,21.13,-2.94,3.01,88,0.19,3.83,10.88,10.24,9.54,9.64,5.36,6.52,-9.51,25.27,3.46,0.01,24.39,3.26,0.01,25.13,-0.55,0.03,25.33,-2.94,0.03,3.46,0.01,3.25,0.01,-0.56,0.03,-2.95,0.03,NGC0253,NED,5
NGC5128,201.36481,-43.01954,S0,0.77,33.8,19.64,3.01,19.64,2.98,19.64,0.45,19.64,-1.25,3.65,106,0.01,2.55,11.23,11.02,9.25,9.12,2.98,2.16,-10.55,23.45,2.99,0.01,23.36,2.98,0.01,9.08,0.41,0.01,8.32,-1.29,0.01,2.98,0.01,2.97,0.01,0.44,0.01,-1.26,0.01,NGC5128,NED,6
NGC0055,3.72192,-39.19775,SBm,0.19,108.9,19.49,5.45,19.49,5.36,19.49,2.8,19.49,0.29,2.1,61,0.09,2.66,9.77,9.39,7.88,8.06,0.19,0.23,-10.11,23.36,5.42,0.01,23.31,5.33,0.01,18.76,2.8,0.01,18.66,0.29,0.01,5.43,0.01,5.3,0.02,2.7,0.03,0.22,0.02,NGC0055,NED,7
MESSIER81,148.88837,69.06528,Sab,0.56,156.7,14.67,3.6,14.67,3.61,14.67,1.84,14.67,0.45,3.65,106,-0.01,1.81,10.99,10.85,8.62,8.44,0.82,0.52,-10.94,16.57,3.6,0.01,16.97,3.6,0.01,10.34,1.82,0.03,10.61,0.44,0.03,3.6,0.01,3.59,0.01,1.79,0.03,0.35,0.03,MESSIER81,NED,8


# Morphology cleaning process

For this sample, LMC & SMC, irregulars (IB, IBm) and very-late type (Sm) must be dropped due to possible miscalculation in bulge fraction from their morphology. For that we defineed the function drop_irregular() to 

In [3]:
def drop_irregulars(strings):
    def criteria(string):
        for i in range(len(string)-1):
            if ((string[i].isupper()) and (string[i + 1] == 'm')) | (string[0]=='I'):
                return False
        return True
    boolean_array = [criteria(string) for string in strings]
    return boolean_array

In [4]:
drop_boolean = drop_irregulars(wxsc['Morph'])
wxsc_no_irr = wxsc[drop_boolean]

In [5]:
np.unique(wxsc_no_irr['Morph'])

E
E-S0
E1pec
S0
S0-a
S?
SABa
SABb
SABc
SABcd
SAd


In [10]:
def keep_first_and_non_capitals(strings):
    def criteria(string):
        result = string[0]  # Initialize the result with the first letter

        # Iterate through the remaining characters in the string
        for char in string[1:]:
            if char.islower() or not char.isalpha():
                result += char

        return result.strip()
    output = [criteria(s) for s in input_strings]
    return output

# Test the function with a list of strings
input_strings = wxsc_no_irr['Morph']
output = keep_first_and_non_capitals(input_strings)
np.unique(output)


array(['E', 'E-0', 'E1pec', 'S0', 'S0-a', 'S?', 'Sa', 'Sab', 'Sb', 'Sbc',
       'Sc', 'Scd', 'Sd', 'Sdm', 'Z'], dtype='<U5')

For sources with E1pec, we will assume an E morphology. For both S? and Z, the morphology will be estimated using the color of presented in WXSC. K-corrections will be applied based on NED redshift.

In [15]:
wm.query_ned(wxsc, ra_column='RAJ2000', dec_column='DEJ2000')

UnitTypeError: Longitude instances require units equivalent to 'rad', so cannot set it to 'deg2'.